In [ ]:
import numpy as np
import matplotlib.pyplot as plt  
%matplotlib inline
# from IPython.display import display, HTML

from pykat import finesse
from pykat.commands import *
pykat.init_pykat_plotting(dpi=90)

from pprint import pprint as pprint

from scipy.optimize import minimize

import os, sys

In [ ]:
# base of configurations
# reference: http://www.gwoptics.org/finesse/examples/aligo_sensitivity.php

basekat = finesse.kat()
basecode = """
# lasers
const Pin 125
const PLO1 1
# spaces
const lx 1.6
const ly 1.55
const Lx 4k
const Ly 4k
const lprc 53
# mirrors
const Tprm 0.03
const Rprm 0.97
const Titm 0.014
const Ritm 0.986
const Tetm 6E-6
const Retm 0.999994
const Tsrm 0.2
const Rsrm 0.8
const Mtm 40 #kg

# carrier laser
l laser $Pin 0 nLaser

# power recycling
m PRM $Rprm $Tprm 90 nLaser nPRM # tuning = 90° towards nLaser
s sBStoPRM $lprc nPRM nBSi

# central beam splitter
bs BS 0.5 0.5 0 45 nBSi nBSr nBSt nBSo
s sBStoYarm $ly nBSr nY0
s sBStoXarm $lx nBSt nX0

# Y arm (perpendicular)
m mYitm $Ritm $Titm 0 nY0 nY1
attr mYitm mass $Mtm
s sY $Ly nY1 nY2
m mYetm $Retm $Tetm 0 nY2 nY3
attr mYetm mass $Mtm

# X arm (parallel)
m mXitm $Ritm $Titm 90 nX0 nX1 # tuning = 90° towards nX0
attr mXitm mass $Mtm
s sX $Lx nX1 nX2
m mXetm $Retm $Tetm 90 nX2 nX3 # tuning = 90° towards nX2
attr mXetm mass $Mtm

# signal recycling
# nBSo to nSRM left disconnected for now
# is connected differently in kat1 and kat2
m SRM $Rsrm $Tsrm 90 nSRM nDark
s sSRMtohdBS 0 nDark nhdBSi # null length space

# (balanced) homodyne detection
# phase of LO s.t. beamsplitter has one constructive, one destructive
l LO1 $PLO1 0 -90 nLO1
bs hdBS 0.5 0.5 0 45 nhdBSi nhdBSr nhdBSt nLO1
qhd qhd1 180 nhdBSr nhdBSt
hd hd1 180 nhdBSr nhdBSt
"""
basekat.parse(basecode)

# pprint((basekat.components, basekat.detectors, basekat.commands))

In [ ]:
def check_tuning(kat):
    """checking IFO tuning by seeing if circulating power in arms is close to 800kW"""
    check_tuning_kat = deepcopy(kat)
    check_tuning_kat.verbose = False
    check_tuning_code="""
    pd initial nLaser*
    pd circPRC nPRM
    pd circX nX1
    pd circY nY1

    pd BSr nBSr
    pd BSt nBSt
    pd BSo nBSo

    noxaxis
    """
    check_tuning_kat.parse(check_tuning_code)
    check_tuning_out = check_tuning_kat.run()

    pprint([(x, '{0:.2f}'.format(check_tuning_out[x].mean())) for x in check_tuning_out.ylabels])    

In [ ]:
def create_kat1(lsrc):
    """configuration 1: connecting nBSo and nSRM with a space
    lsrc in m
    """
    kat1 = deepcopy(basekat)
    kat1code = """
    const lsrc {0}
    s sBStoSRM $lsrc nBSo nSRM
    """.format(lsrc)
    kat1.parse(kat1code)
    
    return kat1


def create_kat2(lsrc, sqz_gain=0.1, sqz_angle=0):
    """configuration 2: connecting nBSo and nSRM with a nle
    lsrc in m, sqz_gain in field dB, sqz_angle in deg
    """
    kat2 = deepcopy(basekat)
    kat2code = """
    const lsrc {0}
    const halflsrc {1}
    s sBStonle $halflsrc nBSo nnle11
    nle nle1 {2} {3} nnle11 nnle12
    s snletoSRM $halflsrc nnle12 nSRM  
    """.format(lsrc, lsrc/2, sqz_gain, sqz_angle)
    kat2.parse(kat2code)
    
    return kat2

In [ ]:
def differential_transfer(ifo, fmin=None, fmax=None):
    """calculates transfer function for ifo object of kat1 or kat2
    for differential shaking of arm spaces
    NB: fsig accounts for vacuum noise
    """
    kat = ifo.kat
    if fmin is None:
        fmin = ifo.fmin
    if fmax is None:
        fmax = ifo.fmax    
    
    transfer_kat = deepcopy(kat)
    transfer_kat.verbose = False
    # initial frequency of 1 for fsig doesn't matter   
    transfer_code = """          
    fsig darm sX 1 0
    fsig darm sY 1 180 # differential tuning
    
    xaxis darm f log {0} {1} 100
    yaxis log abs
    """.format(fmin, fmax)
    transfer_kat.parse(transfer_code)
    transfer_out = transfer_kat.run()    
    
    return transfer_out

In [ ]:
class IFO(object):
    def __init__(self, include_nle, lsrc, title=None, filetag=None,
                 fmin=10, fmax=1e4, sqz_gain=0.1, sqz_angle=0, homodyne_angle=None):
        """instance of kat1 or kat2
        include_nle is flag for whether to include nle in config
        lsrc is space from central BS to SRM
        title is appended to all plot titles
        filetag is appended to all filenames, no .pdf required
        (fmin, fmax) is frequency range for xaxis
        sqz_gain, sqz_angle are values for nle
        homodyne_angle is set after creation if specified
        """
        self.lsrc = lsrc
        if title is None:
            self.title = repr((include_nle, lsrc))        
        else:
            self.title = title
        if filetag is None:
            self.filetag = ".pdf"
        else:
            self.filetag = filetag+".pdf"
        self.fmin = fmin
        self.fmax = fmax
        
        if not include_nle:
            self.kat = create_kat1(lsrc)
        elif include_nle:
            self.kat = create_kat2(lsrc, sqz_gain, sqz_angle)
        else:
            raise ValueError
            
        if homodyne_angle is not None:
            self.kat.LO1.phase = homodyne_angle
            
    def plot_transfer_fns_and_sensitivity(self, fmin=None, fmax=None, show=True):
        """plot transfer functions and noise limited sensitivity"""
        if fmin is None:
            fmin = self.fmin
        if fmax is None:
            fmax = self.fmax
        
        plot_out = differential_transfer(self)
        plot_x = plot_out.x
        plot_noise = plot_out['qhd1']
        plot_signal = plot_out['hd1']

        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8,8))

        ax1.plot(plot_x, plot_noise, label="qhd1")
        ax1.set(title="noise transfer fn - "+self.title,
                xlabel="frequency / Hz", ylabel="noise / au",
                yscale="log", xscale="log")
        ax1.legend()

        ax2.plot(plot_x, plot_signal, label="hd1")
        ax2.set(title="signal transfer fn - "+self.title,
                xlabel="frequency / Hz", ylabel="signal / au",
                yscale="log", xscale="log")
        ax2.legend()

        ax3.plot(plot_x, plot_noise/plot_signal, label="NSR")
        ax3.set(title="QN limited sensitivity - "+self.title,
                xlabel="frequency / Hz", ylabel="strain / ${Hz}^{-1/2}$",
                yscale="log", xscale="log")
        ax3.legend()

        fig.tight_layout()
        fig.savefig("aLIGO_transfer_fns_and_sensitivity"+self.filetag)             

In [ ]:
def combined_sensitivity_plots(filename, *args):
    """combines sensitivity plots for ifo's over same frequency axis
    expects each arg in args to be of the form (ifo, label, fmt)
    if no format specified, use empty string
    """
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8,8), sharex=True)
    
    plot_x = None
    
    for arg in args:
        ifo, label, fmt = arg

        out = differential_transfer(ifo)
        noise = out['qhd1']
        signal = out['hd1']
        if plot_x is None:
            plot_x = out.x
            
        ax1.plot(plot_x, noise, fmt, label="qhd1-"+label)
        ax2.plot(plot_x, signal, fmt, label="hd1-"+label)
        ax3.plot(plot_x, noise/signal, fmt, label="NSR-"+label)
        
    ax1.set(title="noise transfer fn", ylabel="noise",
            yscale="log", xscale="log")   
    ax2.set(title="signal transfer fn", ylabel="signal",
            yscale="log", xscale="log")
    ax3.set(title="QN limited sensitivity",
            xlabel="frequency / Hz", ylabel="strain / ${Hz}^{-1/2}$",
            yscale="log", xscale="log")
    ax1.legend()     
    ax2.legend()
    ax3.legend()
    
    fig.tight_layout()
    fig.savefig(filename)

In [ ]:
def peak_sensitivity(ifo):
    """given a ifo, return peak sensitivity and frequency thereof"""
    opt_out = differential_transfer(ifo)
    opt_nsr = opt_out['qhd1']/opt_out['hd1']

    peak_depth = opt_nsr.min()
    peak_freq = (opt_out.x)[opt_nsr.argmin()]

    return peak_depth, peak_freq

def ifo_args_to_sensitivity(lsrc, sqz_gain, sqz_angle, homodyne_angle):
    """given squeezer gain, squeezer angle, and homodyne angle, return sensitivity"""
    ifo = IFO(1, lsrc, sqz_gain=sqz_gain, sqz_angle=sqz_angle)
    ifo.kat.LO1.phase = homodyne_angle
    
    return peak_sensitivity(ifo)

class HiddenPrints:
    """hides prints in block when used as "with HiddenPrints():"
    https://stackoverflow.com/a/45669280
    """
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

def best_peak_sensitivity(lsrc, init_sqz_gain, init_sqz_angle, init_homodyne_angle,
                          return_values=True, print_values=False):
    """optimise depth of sensitivity at cavity pole for given lsrc
    against squeezer gain, squeezer angle, and homodyne angle
    returns best peak sensitivity and arguments thereof
    """
    log_opt_fn = lambda x, lsrc: np.log(ifo_args_to_sensitivity(lsrc, x[0], x[1], x[2])[0])

    with HiddenPrints():
        result = minimize(log_opt_fn, (init_sqz_gain, init_sqz_angle, init_homodyne_angle), args=lsrc,
                          bounds=((0, None), (-180, 180), (-180, 180)))
    
    if not result["success"]:
        raise ValueError("minimisation didn't converge")
    
    sensitivity = np.exp(result['fun'])
    sensi_args = result['x']
    
    if print_values:
        print("""
for SRC of length: {4}
local best peak sensitivity: {0:.4g}
obtained for:
    squeezer gain = {1:.3g}
    squeezer angle = {2:.3g}
    homodyne angle = {3:.3g}
        """.format(sensitivity, *sensi_args, lsrc))
    
    if return_values:
        return np.exp(result['fun']), result['x']

In [ ]:
check_tuning(basekat)

In [ ]:
fmin, fmax = 10, 1e4

# k is a poor choice of name, given that these are ifo's, not kat's
k11 = IFO(0, 53,   "config. 1, $l_{src}$ = 53 m", "_k11", fmin, fmax)
k12 = IFO(0, 2000, "config. 1, $l_{src}$ = 2 km", "_k12", fmin, fmax)
k21 = IFO(1, 53,   "config. 2, $l_{src}$ = 53 m", "_k21", fmin, fmax)
k22 = IFO(1, 2000, "config. 2, $l_{src}$ = 2 km", "_k22", fmin, fmax)

combined_sensitivity_plots("aLIGO_transfer_fns_and_sensitivity_comparison.pdf",
                           (k11, "no nle-short SRC", ""),
                           (k12, "no nle-long SRC", "--"),
                           (k21, "nle-short SRC", "g"),
                           (k22, "nle-long SRC", "--"))

In [ ]:
# # for plotting separately
# k11.plot_transfer_fns_and_sensitivity()
# k12.plot_transfer_fns_and_sensitivity()
# k21.plot_transfer_fns_and_sensitivity()
# k22.plot_transfer_fns_and_sensitivity()

In [ ]:
k21_local_min = best_peak_sensitivity(53, 0.1, 0, -90, print_values=True)
k22_local_min = best_peak_sensitivity(2000, 0.1, 0, -90, print_values=True)

# combined plotting, but with locally optimum parameters
# for config. 1 can just use k11 and k12 again
k21_opt = IFO(1, 53, sqz_gain=k21_local_min[1][0],
              sqz_angle=k21_local_min[1][1], homodyne_angle=k21_local_min[1][2])
k22_opt = IFO(1, 2000, sqz_gain=k22_local_min[1][0],
              sqz_angle=k22_local_min[1][1], homodyne_angle=k22_local_min[1][2])

combined_sensitivity_plots("aLIGO_optimum_sensitivity_comparison.pdf",
                           (k11, "no nle-short SRC", ""),
                           (k12, "no nle-long SRC", "--"),
                           (k21_opt, "nle-short SRC-optimised", "c"),
                           (k22_opt, "nle-long SRC-optimised", "--y"))

In [ ]:
# for comparing against analytics
analytics_lengths = 53, 539.75, 1026.5, 1513.25, 2000

# todo to-do: actually use the values from analytics (ask Vaishali)
analytics_sqz_gain = 0.1
analytics_sqz_angle = 0
analytics_homodyne_angle = -90

analytics_ifos = ((IFO(1, l,
                       sqz_gain=analytics_sqz_gain,
                       sqz_angle=analytics_sqz_angle,
                       homodyne_angle=analytics_homodyne_angle),
                   "nle-$l_{{src}}$ = {0:.3f}".format(l),
                   "") for l in analytics_lengths)

combined_sensitivity_plots("aLIGO_sensitivities_to_compare_to_analytics.pdf",
                           *analytics_ifos)